In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.1 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # search space created above

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy # TPE Sampler triggers Bayesian Search --> decided hyperparameter value based on past values
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2026-01-05 05:28:41,962] A new study created in memory with name: no-name-d87f0016-c9c2-4530-8ab6-189e4f263869
[I 2026-01-05 05:28:43,426] Trial 0 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 129, 'max_depth': 7}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-01-05 05:28:44,476] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 72, 'max_depth': 11}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-01-05 05:28:45,838] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 99, 'max_depth': 18}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-01-05 05:28:47,844] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 142, 'max_depth': 20}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-01-05 05:28:50,987] Trial 4 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 147, 'max_depth': 4}. Best is trial 0 with value: 0.78026070

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 125, 'max_depth': 18}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)  # based on above final parameter values

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


Samplers In Optuna

In [9]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [10]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2026-01-05 05:29:47,038] A new study created in memory with name: no-name-61c23615-9755-482b-bb57-d8b0fff7782e
[I 2026-01-05 05:29:47,622] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-01-05 05:29:48,531] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-01-05 05:29:48,850] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-01-05 05:29:49,477] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-01-05 05:29:50,084] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


Optuna Visualizations

In [13]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [14]:
# 1. Optimization History
plot_optimization_history(study).show()

In [15]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [16]:
# 3. Slice Plot
plot_slice(study).show()

In [17]:
# 4. Contour Plot
plot_contour(study).show()

In [18]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

**Optimizing Multiple ML Models**

In [19]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [20]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [22]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2026-01-05 05:45:24,123] A new study created in memory with name: no-name-384a11b2-a1b9-4eca-b4d6-93c152161daf
[I 2026-01-05 05:45:25,176] Trial 0 finished with value: 0.7653631284916201 and parameters: {'classifier': 'RandomForest', 'n_estimators': 104, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: 0.7653631284916201.
[I 2026-01-05 05:45:27,725] Trial 1 finished with value: 0.7616387337057727 and parameters: {'classifier': 'RandomForest', 'n_estimators': 256, 'max_depth': 19, 'min_samples_split': 10, 'min_samples_leaf': 7, 'bootstrap': True}. Best is trial 0 with value: 0.7653631284916201.
[I 2026-01-05 05:45:33,552] Trial 2 finished with value: 0.756052141527002 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 300, 'learning_rate': 0.010381035631427164, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7653631284916201.
[I 2026-01-05 05:45:36,355] Trial 3

In [23]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.14021823876668812, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [24]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.765363,2026-01-05 05:45:24.127353,2026-01-05 05:45:25.176332,0 days 00:00:01.048979,NaN,True,RandomForest,NaN,NaN,NaN,7.0,1.0,3.0,104.0,COMPLETE
1,1,0.761639,2026-01-05 05:45:25.177810,2026-01-05 05:45:27.725743,0 days 00:00:02.547933,NaN,True,RandomForest,NaN,NaN,NaN,19.0,7.0,10.0,256.0,COMPLETE
2,2,0.756052,2026-01-05 05:45:27.726997,2026-01-05 05:45:33.552258,0 days 00:00:05.825261,NaN,NaN,GradientBoosting,NaN,NaN,0.010381,17.0,9.0,2.0,300.0,COMPLETE
3,3,0.757914,2026-01-05 05:45:33.553619,2026-01-05 05:45:36.355282,0 days 00:00:02.801663,NaN,NaN,GradientBoosting,NaN,NaN,0.017699,6.0,7.0,4.0,290.0,COMPLETE
4,4,0.767225,2026-01-05 05:45:36.356435,2026-01-05 05:45:36.395106,0 days 00:00:00.038671,0.207600,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
5,5,0.737430,2026-01-05 05:45:36.396166,2026-01-05 05:45:39.883113,0 days 00:00:03.486947,NaN,NaN,GradientBoosting,NaN,NaN,0.081312,14.0,6.0,7.0,225.0,COMPLETE
6,6,0.733706,2026-01-05 05:45:39.884059,2026-01-05 05:45:39.914964,0 days 00:00:00.030905,1.948726,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
7,7,0.772812,2026-01-05 05:45:39.915947,2026-01-05 05:45:40.215959,0 days 00:00:00.300012,NaN,False,RandomForest,NaN,NaN,NaN,17.0,5.0,4.0,56.0,COMPLETE
8,8,0.761639,2026-01-05 05:45:40.216962,2026-01-05 05:45:41.366175,0 days 00:00:01.149213,NaN,True,RandomForest,NaN,NaN,NaN,6.0,2.0,5.0,184.0,COMPLETE
9,9,0.772812,2026-01-05 05:45:41.368610,2026-01-05 05:45:41.424850,0 days 00:00:00.056240,3.338518,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [25]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,10
RandomForest,7
GradientBoosting,3


In [26]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.750466
RandomForest,0.765629
SVM,0.767598


In [27]:
# 1. Optimization History
plot_optimization_history(study).show()